In [1]:
!pip install julep

In [8]:
import os

import yaml
from julep import Julep

api_key = os.getenv("JULEP_API_KEY")

julep = Julep(api_key=api_key, environment="dev")

In [9]:
agent = julep.agents.create(
    name="Julep Email Assistant",
    about=(
        "You are an agent that handles emails for julep users."
        + " Julep is a platform for creating kick-ass AI agents."
    ),
    model="gpt-4o",
    default_settings={"temperature": 0.2},
)

In [10]:
agent.id

'231366f8-cdc8-423a-a1c6-72d4a300675f'

In [11]:
mailgun_password = os.getenv("MAILGUN_PASSWORD")

task_def = yaml.safe_load(f"""
name: Julep Email Assistant

input_schema:
  type: object
  properties:
    from:
      type: string
    to:
      type: string
    subject:
      type: string
    body:
      type: string

tools:
- name: send_email
  integration:
    provider: email
    setup:
      host: smtp.mailgun.org
      password: {mailgun_password}
      port: 587
      user: postmaster@email.julep.ai

- name: search_docs
  system:
    resource: agent
    subresource: doc
    operation: search
    
main:
- prompt: |-
    You are {{{{ agent.name }}}}. {{{{ agent.about }}}}

    A user with email address {{{{ _.from }}}} has sent the following inquiry:
    ------
      Subject: {{{{ _.subject }}}}

      {{{{ _.body }}}}
    ------

    Can you generate a query to search the documentation based on this email?
    Just respond with the query as is and nothing else.

  unwrap: true

- tool: search_docs
  arguments:
    agent_id: "'{agent.id}'"
    text: _
    
- prompt: |-
    You are {{{{ agent.name }}}}. {{{{ agent.about }}}}

    A user with email address {{{{ inputs[0].from }}}} has sent the following inquiry:
    ------
      Subject: {{{{ inputs[0].subject }}}}

      {{{{ inputs[0].body }}}}
    ------

    Here are some possibly relevant snippets from the julep documentation:
    {{% for doc in _.docs %}}
      {{% for snippet in doc.snippets %}}
        {{{{ snippet.content }}}}
      {{% endfor %}}
    {{% endfor %}}
    ========

    Based on the above info, craft an email body to respond with as a json object.
    The json object must have `subject` and `body` fields.
  response_format:
    type: json_object
    
  unwrap: true
  
- evaluate:
    subject: "load_json(_.split('```json')[1].split('```')[0])['subject']"
    body: "load_json(_.split('```json')[1].split('```')[0])['body']"
    
- tool: send_email
  arguments:
    body: _.body
    from: "'postmaster@email.julep.ai'"
    subject: _.subject
    to: inputs[0]['from']
""")

In [12]:
task = julep.tasks.create(
    agent_id=agent.id,
    **task_def,
)

In [13]:
task.id

'a942a86d-dfcc-4abd-a8e7-0f502a2e4c67'

In [14]:
execution = julep.executions.create(
    task_id=task.id,
    input={"from": "diwank@julep.ai", "to": "help@agents.new", "subject": "what's up", "body": "sup"},
)

In [15]:
julep.executions.get(execution.id)

Execution(id='54c1b4d2-c036-4b22-af8c-8c8a81fe41ad', created_at=datetime.datetime(2024, 10, 7, 14, 15, 15, 575516, tzinfo=datetime.timezone.utc), input={'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up", 'to': 'help@agents.new'}, status='running', task_id='a942a86d-dfcc-4abd-a8e7-0f502a2e4c67', updated_at=datetime.datetime(2024, 10, 7, 14, 15, 16, 717572, tzinfo=datetime.timezone.utc), error=None, metadata={}, output=None)

In [16]:
execution_transitions = julep.executions.transitions.list(
    execution_id=execution.id).items

for transition in reversed(execution_transitions):
    print("Type: ", transition.type)
    print("output: ", transition.output)
    print("-" * 100)

Type:  init
output:  {'body': 'sup', 'from': 'diwank@julep.ai', 'subject': "what's up", 'to': 'help@agents.new'}
----------------------------------------------------------------------------------------------------
Type:  step
output:  "what's up" site:julep.ai/docs
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'docs': [], 'time': 0.007443666458129883}
----------------------------------------------------------------------------------------------------
Type:  step
output:  ```json
{
  "subject": "Hello!",
  "body": "Hi there! How can I assist you today? If you have any questions or need help with Julep, feel free to let me know!"
}
```
----------------------------------------------------------------------------------------------------
Type:  step
output:  {'body': 'Hi there! How can I assist you today? If you have any questions or need help with Julep, feel free to let me know!', 'subject': 'Hello!'}
-----------